In [78]:
import requests
import json
from sentence_transformers import SentenceTransformer

#from lyrics_api import *

In [81]:
def getLyrics(artistName, trackName):

    #CREATE THE URL
    baseUrl = "https://api.musixmatch.com/ws/1.1/"

    apiKey = "&apikey=bd88be6fb629c354c5ca281296ac8c47"

    lyricsMatcher = "matcher.lyrics.get"

    formatUrl = "?format=json&callback=callback"

    artistSearch = "&q_artist="

    trackSearch = "&q_track="

    #MAKE THE API CALL
    apiCall = baseUrl + lyricsMatcher + formatUrl + artistSearch + artistName + trackSearch + trackName + apiKey

    request = requests.get(apiCall)

    data = request.json()

    data = data['message']['body']

    lyrics = data['lyrics']['lyrics_body']
    
    #PROCESS THE DATA
    lyrics = lyrics.split('\n')
    
    lyrics = list(filter(lambda a: a != '', lyrics))
    
    lyrics = lyrics[:-2]
    
    #output = ""
    
    '''
    for sentence in lyrics:
        output = output + sentence + ". "
        
    output = output[:-1]
    '''
    
    return lyrics



In [82]:
lyrics = getLyrics("FIDLAR", "FSU")

lyrics

["I've been waking up",
 'Getting so fucked up',
 "Yeah, I'm all fucked up",
 'Tryna fuck shit up',
 "I've been waking up",
 'Wanna fuck you up',
 'Wanna fuck you up',
 'I wanna fuck you up, ah',
 "I've been waking up",
 'Getting so fucked up',
 'Tryna fuck shit up',
 "Yeah, I'm all fucked up, ah",
 "I've been waking up",
 'Wanna fuck you up',
 'Wanna fuck you up',
 'I wanna fuck you up']

In [86]:
def getEmbedding(lyrics):
    model = SentenceTransformer('all-MiniLM-L6-v2')

    embeddings = model.encode(lyrics)

    return embeddings

In [87]:
embeddings

array([[ 0.00114902,  0.00313718,  0.03125548, ..., -0.01650619,
        -0.01244194, -0.06759863],
       [-0.04123046,  0.05351889,  0.03241228, ..., -0.0114714 ,
         0.02905795, -0.04907828],
       [ 0.00386012, -0.05138487,  0.06638323, ..., -0.07088796,
         0.04141023, -0.09461488],
       ...,
       [-0.04801308,  0.04056778,  0.02719494, ..., -0.00431593,
         0.01439665, -0.09861775],
       [-0.04801308,  0.04056778,  0.02719494, ..., -0.00431593,
         0.01439665, -0.09861775],
       [-0.08766278,  0.04322638,  0.0267636 , ..., -0.00296094,
         0.0123751 , -0.11467296]], dtype=float32)